# Task 2 – Analytical Chatbot (RAG & Explanation)

This notebook demonstrates a Retrieval-Augmented Generation (RAG) chatbot. The code is organized logically: intent understanding, news retrieval, vector search, trend analysis, and final explanation generation. Each step is written clearly for readability.

In [1]:
!pip install --upgrade \
langchain \
langchain-core \
langchain-community \
langchain-groq \
yfinance \
newsapi-python \
sentence-transformers \
faiss-cpu


In [2]:
import yfinance as yf
import numpy as np
import re

from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer
import faiss

# ✅ UPDATED LangChain imports
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

from langchain_groq import ChatGroq


In [3]:
GROQ_API_KEY = "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"
NEWS_API_KEY = "d47bb4f19d6340d1b822aba510ff1b83"

newsapi = NewsApiClient(api_key=NEWS_API_KEY)


In [4]:
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.1-8b-instant",
    temperature=0.2
)


In [5]:
intent_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    Identify the stock ticker and intent from the user query.

    Query: {query}

    Respond strictly in this format:
    Ticker:
    Intent:
    """
)

intent_chain = RunnableSequence(intent_prompt, llm)

def intent_agent(query):
    return intent_chain.invoke({"query": query})


In [6]:
ticker_map = {
    "apple": "AAPL",
    "microsoft": "MSFT",
    "google": "GOOGL",
    "amazon": "AMZN",
    "tesla": "TSLA"
}

def normalize_ticker(query):
    query = query.lower()
    for name, ticker in ticker_map.items():
        if name in query or ticker.lower() in query:
            return ticker
    return None


In [7]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def research_agent(company):
    articles = newsapi.get_everything(
        q=company,
        language="en",
        sort_by="relevancy",
        page_size=6
    )

    texts = []
    for article in articles["articles"]:
        if article["title"] and article["description"]:
            texts.append(article["title"] + ". " + article["description"])

    embeddings = embedder.encode(texts)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))

    return texts, index


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
def rag_retrieve(query, texts, index):
    query_embedding = embedder.encode([query])
    _, idx = index.search(np.array(query_embedding), 1)
    return texts[idx[0][0]]


In [9]:
def trend_agent(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="7d")

    if len(data) < 2:
        return "no clear trend"

    if data["Close"][-1] > data["Close"][0]:
        return "uptrend"
    else:
        return "downtrend"


In [10]:
explain_prompt = PromptTemplate(
    input_variables=["ticker", "trend", "news"],
    template="""
    Explain in simple language why the stock moved.

    Stock: {ticker}
    Trend: {trend}
    Related News: {news}

    Explanation:
    """
)

explain_chain = RunnableSequence(explain_prompt, llm)


In [11]:
def analytical_chatbot(query):
    # Intent Agent
    intent_output = intent_agent(query)
    ticker = normalize_ticker(query)

    if not ticker:
        return "Sorry, I could not identify the stock."

    company = [k for k, v in ticker_map.items() if v == ticker][0]

    # Research Agent
    texts, index = research_agent(company)
    context = rag_retrieve(query, texts, index)

    # Trend Agent
    trend = trend_agent(ticker)

    # Explanation Agent
    explanation = explain_chain.invoke({
        "ticker": ticker,
        "trend": trend,
        "news": context
    })

    return explanation


In [13]:
print(analytical_chatbot("Why did Apple stock drop?"))



/tmp/ipython-input-2986594822.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data["Close"][-1] > data["Close"][0]:


content="The stock of AAPL (Apple) moved because of a positive news about a deal between Apple and Google. Apple has chosen Google's AI technology, called Gemini, to improve its virtual assistant Siri. This deal is expected to give Siri new and better features, making it more useful and personalized for users.\n\nThink of it like this: Imagine you're buying a new smartphone, and you hear that the company is going to make it even better and more useful with new features. You would be excited and want to buy the phone, right? That's what's happening with Apple's stock. Investors are excited about the potential of this new deal and are buying more shares, which is making the stock go up." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 136, 'total_tokens': 279, 'completion_time': 0.293173147, 'completion_tokens_details': None, 'prompt_time': 0.019759661, 'prompt_tokens_details': None, 'queue_time': 0.006591785, 'total_time': 0.312932808},